# Exploratory data analysis: Wine quality data

In [17]:
# Required modules
import pandas as pd
import numpy as np
import seaborn as sns
import scipy
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [22]:
# Import raw data
raw_data = pd.read_csv("../data/winequalityN.csv")
raw_data.head()

x = raw_data.drop(['quality'], axis=1)
y = raw_data.quality

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=128)

## Data cleaning

### Missing values

In [23]:
total = x_train.isnull().sum().sort_values(ascending=False)
percent = (x_train.isnull().sum() / x_train.isnull().count()).sort_values(
    ascending=False
)
missing_data = pd.concat([total, percent * 100], axis=1, keys=["Total", "Percent"])
missing_data

,Total,Percent
volatile acidity,7,0.134693
pH,7,0.134693
fixed acidity,6,0.115451
sulphates,4,0.076967
citric acid,2,0.038484
chlorides,2,0.038484
residual sugar,1,0.019242
type,0,0.000000
free sulfur dioxide,0,0.000000
total sulfur dioxide,0,0.000000


## Data exploration

### Understanding the target variable

In [8]:
print("Skewness: %f" % raw_data['quality'].skew())
print("Kurtosis: %f" % raw_data['quality'].kurt())

Skewness: 0.189623
Kurtosis: 0.232322
